In [1]:
# pd.set_option("display.max_columns", None)
# pd.reset_option('display.max_columns')
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

Baseline Scenario
- Uncomment Code Below For Individual scenario runs (rather than full model MP8-Basic, MP9-Moderate, MP10-Advanced)

In [2]:
import os

# import from cmu-tare-model package
from config import PROJECT_ROOT

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

print(f"PROJECT_ROOT (from config.py): {PROJECT_ROOT}")

# Construct the absolute path to the .py file
relative_path = os.path.join("cmu_tare_model", "model_scenarios", "tare_baseline_v2_1.ipynb")
file_path = os.path.join(PROJECT_ROOT, relative_path)

# On Windows, to avoid any path-escape quirks, convert backslashes to forward slashes
file_path = file_path.replace("\\", "/")

print(f"Running file: {file_path}")

# %run magic command to run a .py file and import variables into the current IPython session
# # If your path has spaces, wrap it in quotes:
%run -i {file_path} # If your path has NO spaces, no quotes needed.

# iPthon magic command to run a .py file and import variables into the current IPython session
# Now run it, importing variables into your current IPython session
# from IPython import get_ipython
# get_ipython().run_line_magic('run', f'-i {file_path}')  # If your path has NO spaces, no quotes needed.

# # exec() function to run a .py file and import variables into the current IPython session
# with open(file_path) as f:
#     code = compile(f.read(), file_path, 'exec')
#     exec(code)

print("Baseline Scenario - Model Run Complete")

Project root directory: c:\users\14128\research\cmu-tare-model
PROJECT_ROOT (from config.py): c:\users\14128\research\cmu-tare-model
Running file: c:/users/14128/research/cmu-tare-model/cmu_tare_model/model_scenarios/tare_baseline_v2_1.ipynb

-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------
      
Project root directory: c:\users\14128\research\cmu-tare-model
Result outputs will be exported here: c:\users\14128\research\cmu-tare-model\cmu_tare_model\output_results


Retrieved data for filename: baseline_metadata_and_annual_r

## Dataframe for Electric Resistance Cooking (MP7)

In [3]:
print(f"""
====================================================================================================================================================================
We assume the use of Electric Resistance (MP7) rather than Induction (MP8).
Electric Resistance is significantly cheaper and only slightly less efficient than Induction.
====================================================================================================================================================================
""")

# Measure Package 7
menu_mp = 7
input_mp = 'upgrade07'

filename = "upgrade07_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

# Fix DtypeWarning error in columns 'in.neighbors' and 'in.geometry_stories_low_rise'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_mp7 = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id' (unique identifier)
print(f"DATAFRAME SIZE before applying any filters: {df_euss_am_mp7.shape}")


# Filter for occupied homes
occupancy_filter = df_euss_am_mp7['in.vacancy_status'] == 'Occupied'
df_euss_am_mp7 = df_euss_am_mp7.loc[occupancy_filter]
print(f"DATAFRAME SIZE after filtering for 'Occupied' homes: {df_euss_am_mp7.shape}")

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_mp7['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_mp7 = df_euss_am_mp7.loc[house_type_filter]
print(f"DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: {df_euss_am_mp7.shape}")

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_mp7['in.state'].eq(input_state)
    df_euss_am_mp7 = df_euss_am_mp7.loc[state_filter]

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_mp7['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_mp7 = df_euss_am_mp7.loc[city_filter]

# Display the filtered dataframe
print(f"DATAFRAME SIZE after applying geographic filter: {df_euss_am_mp7.shape}")
print(df_euss_am_mp7)


We assume the use of Electric Resistance (MP7) rather than Induction (MP8).
Electric Resistance is significantly cheaper and only slightly less efficient than Induction.

Retrieved data for filename: upgrade07_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\upgrade07_metadata_and_annual_results.csv


DATAFRAME SIZE before applying any filters: (548916, 346)
DATAFRAME SIZE after filtering for 'Occupied' homes: (482597, 346)
DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: (331531, 346)
You chose to filter for: PA
You chose to analyze all of state: PA
DATAFRAME SIZE after applying geographic filter: (15651, 346)
         upgrade      weight  applicability  in.sqft  \
bldg_id                                                
119            7  242.131013           True   2152.0   
122            7  242.131013           True   2176.0   
1

## Dataframe used for other end-uses (MP8)

In [4]:
# Print debugging information
print_debug = True

In [5]:
# Measure Package 8
menu_mp = 8
input_mp = 'upgrade08'
scenario_name = 'Basic-BAU'
cost_scenario = 'BAU Costs'
grid_scenario = 'Current Electricity Grid'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")

filename = "upgrade08_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

# Fix DtypeWarning error in columns 'in.neighbors' and 'in.geometry_stories_low_rise'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_mp8 = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id' (unique identifier)
print(f"DATAFRAME SIZE before applying any filters: {df_euss_am_mp8.shape}")

# Filter for occupied homes
occupancy_filter = df_euss_am_mp8['in.vacancy_status'] == 'Occupied'
df_euss_am_mp8 = df_euss_am_mp8.loc[occupancy_filter]
print(f"DATAFRAME SIZE after filtering for 'Occupied' homes: {df_euss_am_mp8.shape}")

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_mp8['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_mp8 = df_euss_am_mp8.loc[house_type_filter]
print(f"DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: {df_euss_am_mp8.shape}")

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_mp8['in.state'].eq(input_state)
    df_euss_am_mp8 = df_euss_am_mp8.loc[state_filter]

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_mp8['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_mp8 = df_euss_am_mp8.loc[city_filter]

# Display the filtered dataframe
# Display the filtered dataframe
print(f"DATAFRAME SIZE after applying geographic filter: {df_euss_am_mp8.shape}")
print(df_euss_am_mp8)


MODEL SCENARIO
Scenario Basic-BAU:
Basic Retrofit: Measure Package 8
BAU Costs
Current Electricity Grid

Retrieved data for filename: upgrade08_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\upgrade08_metadata_and_annual_results.csv


DATAFRAME SIZE before applying any filters: (548916, 347)
DATAFRAME SIZE after filtering for 'Occupied' homes: (482597, 347)
DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: (331531, 347)
You chose to filter for: PA
You chose to analyze all of state: PA
DATAFRAME SIZE after applying geographic filter: (15651, 347)
         upgrade      weight  applicability  in.sqft  \
bldg_id                                                
119            8  242.131013           True   2152.0   
122            8  242.131013           True   2176.0   
150            8  242.131013           True   1690.0   
153        

# Project Future Energy Consumption

In [6]:
print(F"""
====================================================================================================================================================================
LOAD EUSS DATA FOR MEASURE PACKAGE {menu_mp} (MP{menu_mp})
====================================================================================================================================================================
You'll notice that the number of rows differs from df_euss_am_mp7 and df_euss_am_mp8.
      - df_euss_am_baseline_home has fewer rows (representative dwelling units) because a tech filter was applied. 
      - df_euss_am_mp8_home will have the same number of rows as df_euss_am_baseline_home after df_enduse_compare function is run.
      - df_enduse_compare function performs an inner merge on the two dataframes, keeping only the rows that are present in both dataframes.
====================================================================================================================================================================
df_euss_am_mp8_home will be created by running the df_enduse_compare function (contains post-retrofit consumption data for the entire home in 2024).
process_euss_data.py file contains the function definition.
      
def df_enduse_compare(df_mp: pd.DataFrame, 
                      input_mp: str, 
                      menu_mp: int, 
                      df_baseline: pd.DataFrame, 
                      df_cooking_range: pd.DataFrame
) -> pd.DataFrame:

""")

# df_enduse_compare(df_mp, category, df_baseline):
df_euss_am_mp8_home = df_enduse_compare(
    df_mp = df_euss_am_mp8,
    input_mp=input_mp,
    menu_mp=menu_mp,
    df_baseline = df_euss_am_baseline_home,
    df_cooking_range=df_euss_am_mp7,
    )



LOAD EUSS DATA FOR MEASURE PACKAGE 8 (MP8)
You'll notice that the number of rows differs from df_euss_am_mp7 and df_euss_am_mp8.
      - df_euss_am_baseline_home has fewer rows (representative dwelling units) because a tech filter was applied. 
      - df_euss_am_mp8_home will have the same number of rows as df_euss_am_baseline_home after df_enduse_compare function is run.
      - df_enduse_compare function performs an inner merge on the two dataframes, keeping only the rows that are present in both dataframes.
df_euss_am_mp8_home will be created by running the df_enduse_compare function (contains post-retrofit consumption data for the entire home in 2024).
process_euss_data.py file contains the function definition.
      
def df_enduse_compare(df_mp: pd.DataFrame, 
                      input_mp: str, 
                      menu_mp: int, 
                      df_baseline: pd.DataFrame, 
                      df_cooking_range: pd.DataFrame
) -> pd.DataFrame:



Applying combined vali

In [7]:
print(f"""
====================================================================================================================================================================
PROJECT FUTURE ENERGY CONSUMPTION
====================================================================================================================================================================
project_future_energy_consumption.py file was used to calculate/project the annual energy consumption 
for each home in the dataframe.
      
def project_future_consumption(
    df: pd.DataFrame, 
    menu_mp: int,
    base_year: int = 2024
) -> Tuple[pd.DataFrame, pd.DataFrame]:

""")
from cmu_tare_model.energy_consumption_and_metadata.project_future_energy_consumption import *

# Make copies to avoid modifying the original dataframes
df_euss_am_mp8_home = df_euss_am_mp8_home.copy()
df_mp8_consumption = df_euss_am_mp8_home.copy()

# Project Future Energy Consumption
df_euss_am_mp8_home, df_mp8_consumption = project_future_consumption(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    base_year=2024
    )

print(f"""
====================================================================================================================================================================
DATAFRAME FOR MEASURE PACKAGE {menu_mp} (MP{menu_mp}):

{df_euss_am_mp8_home}
     
DATAFRAME (Consumption Cols): df_mp8_consumption
      
{df_mp8_consumption}

""")


PROJECT FUTURE ENERGY CONSUMPTION
project_future_energy_consumption.py file was used to calculate/project the annual energy consumption 
for each home in the dataframe.
      
def project_future_consumption(
    df: pd.DataFrame, 
    menu_mp: int,
    base_year: int = 2024
) -> Tuple[pd.DataFrame, pd.DataFrame]:



Determining valid homes for calculations:
Measure package calculation for heating:
  - 12266 homes have valid baseline data
  - 15651 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Measure package calculation for waterHeating:
  - 14999 homes have valid baseline data
  - 15651 homes will receive retrofits
  - 14999 homes have both valid data AND will receive retrofits
  - 652 homes excluded (values will be NaN)
Measure package calculation for clothesDrying:
  - 14743 homes have valid baseline data
  - 15651 homes will receive retrofits
  - 14743 homes have both valid data AND will re

In [8]:
if print_debug:
    from cmu_tare_model.utils.create_sample_df import *

    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_heating', 'valid_fuel_heating', 'valid_tech_heating', 'include_heating', 'heating_consumption']
    )
    print(df_sample_heating)

    # Create a sample dataframe for the waterHeating category
    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['waterHeating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'valid_tech_waterHeating', 'include_waterHeating', 'waterHeating_consumption']
    )
    print(df_sample_waterHeating)

    # Create a sample dataframe for the clothesDrying category
    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['clothesDrying'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_consumption']
    )
    print(df_sample_clothesDrying)

    # Create a sample dataframe for the cooking category
    df_sample_cooking = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_consumption']
    )
    print(df_sample_cooking)

        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

# PUBLIC IMPACTS: CLIMATE AND HEALTH DAMAGES
## Scenarios: No IRA and IRA-Reference

In [9]:
print("""
====================================================================================================================================================================
PUBLIC IMPACTS: DAMAGES FROM CLIMATE AND HEALTH-RELATED EMISSIONS
====================================================================================================================================================================

""")
from cmu_tare_model.public_impact.calculate_lifetime_climate_impacts_sensitivity import *
from cmu_tare_model.public_impact.calculate_lifetime_health_impacts_sensitivity import *

# Make copies from scenario consumption to keep df smaller
print("\n", "Creating dataframe to store marginal damages calculations ...")
# Climate damages: No IRA and IRA-Reference
df_mp8_noIRA_damages_climate = df_mp8_consumption.copy()
df_mp8_IRA_damages_climate = df_mp8_consumption.copy()

# Health damages: No IRA and IRA-Reference
df_mp8_noIRA_damages_health = df_mp8_consumption.copy()
df_mp8_IRA_damages_health = df_mp8_consumption.copy()



PUBLIC IMPACTS: DAMAGES FROM CLIMATE AND HEALTH-RELATED EMISSIONS



 Creating dataframe to store marginal damages calculations ...


## Future Climate Damages: No IRA and IRA-Reference

In [10]:
print("""
========== SCENARIO: No Inflation Reduction Act ==========
""")
df_euss_am_mp8_home, df_mp8_noIRA_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='No Inflation Reduction Act', 
    df_baseline_damages=df_baseline_damages_climate,
    verbose=False  # Add this parameter
    )

df_euss_am_mp8_home, df_mp8_noIRA_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='No Inflation Reduction Act', 
    df_baseline_damages=df_baseline_damages_health,
    debug=False,
    verbose=False  # Add this parameter
    )


print("""
========== SCENARIO: Inflation Reduction Act (AEO2023 Reference Case) ==========
""")
df_euss_am_mp8_home, df_mp8_IRA_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='AEO2023 Reference Case', 
    df_baseline_damages=df_baseline_damages_climate,
    verbose=False  # Add this parameter
    )


df_euss_am_mp8_home, df_mp8_IRA_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='AEO2023 Reference Case', 
    df_baseline_damages=df_baseline_damages_health,
    debug=False,
    verbose=False  # Add this parameter
    )


print(f"""  
====================================================================================================================================================================
Post-Retrofit (MP8) Marginal Damages: WHOLE-HOME
Scenario: No Inflation Reduction Act and AEO2023 Reference Case
====================================================================================================================================================================
calculate_emissions_damages.py file contains the definition for the calculate_marginal_damages function.
Additional information on emissions and damage factor lookups can be found in the calculate_emissions_damages.py file as well. 
      
CLIMATE DAMAGES: No IRA and IRA-Reference
--------------------------------------------------------
Climate Damages (No IRA): df_mp8_noIRA_damages_climate
{df_mp8_noIRA_damages_climate}

Climate Damages (IRA): df_mp8_IRA_damages_climate
{df_mp8_IRA_damages_climate}

HEALTH DAMAGES: No IRA and IRA-Reference
--------------------------------------------------------
Health Damages (No IRA): df_mp8_noIRA_damages_health
{df_mp8_noIRA_damages_health}

Health Damages (IRA): df_mp8_IRA_damages_health
{df_mp8_IRA_damages_health}

SUMMARY DATAFRAME FOR MP8: df_euss_am_mp8_home
{df_euss_am_mp8_home}
====================================================================================================================================================================
""")


========== SCENARIO: No Inflation Reduction Act ==========

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              

Verifying masking for all calculated columns:
-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              

Verifying masking for all calculated columns:

==

In [11]:
if print_debug:
    from cmu_tare_model.utils.create_sample_df import *

    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_heating', 'valid_fuel_heating', 'include_heating', 'heating_lifetime_damages']
    )
    print(df_sample_heating)

    # Create a sample dataframe for the waterHeating category
    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['waterHeating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'include_waterHeating', 'waterHeating_lifetime_damages']
    )
    print(df_sample_waterHeating)

    # Create a sample dataframe for the clothesDrying category
    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['clothesDrying'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_lifetime_damages']
    )
    print(df_sample_clothesDrying)

    # Create a sample dataframe for the cooking category
    df_sample_cooking = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_damages']
    )
    print(df_sample_cooking)

        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

## Future Annual Fuel Costs: No IRA and IRA-Reference

In [ ]:
print("""  
====================================================================================================================================================================
PRIVATE IMPACTS: LIFETIME FUEL COSTS
====================================================================================================================================================================
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
====================================================================================================================================================================
""")
from cmu_tare_model.private_impact.calculate_lifetime_fuel_costs import *

print("\n", "Creating dataframe to store annual fuel cost calculations ...")
# Annual fuel costs: No IRA and IRA-Reference
df_mp8_fuelCosts_noIRA = df_mp8_consumption.copy()
df_mp8_fuelCosts_IRA = df_mp8_consumption.copy()

# # Make copies to prevent overwriting the original dataframe and compare the differences
# df_euss_am_mp8_home = df_euss_am_mp8_home.copy()
# df_mp8_fuelCosts = df_euss_am_mp8_home.copy()

In [ ]:
print("""
========== SCENARIO: No Inflation Reduction Act ==========
""")
df_euss_am_mp8_home, df_mp8_noIRA_fuelCosts = calculate_lifetime_fuel_costs(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act'
    )

print("""
========== SCENARIO: Inflation Reduction Act (AEO2023 Reference Case) ==========
""")
df_euss_am_mp8_home, df_mp8_IRA_fuelCosts = calculate_lifetime_fuel_costs(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case'
    )


print(f"""  
====================================================================================================================================================================
Creating Dataframes for Lifetime Fuel Costs ...

FUEL COSTS (No IRA): df_mp8_noIRA_fuelCosts
{df_mp8_noIRA_fuelCosts}

FUEL COSTS (IRA): df_mp8_IRA_fuelCosts
{df_mp8_IRA_fuelCosts}

SUMMARY DATAFRAME FOR MP8: df_euss_am_mp8_home
{df_euss_am_mp8_home}

====================================================================================================================================================================
""")

In [ ]:
if print_debug:
    from cmu_tare_model.utils.create_sample_df import *

    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # df_sample_climate

    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_heating', 'valid_fuel_heating', 'include_heating', 'heating_lifetime_fuelCost']
    )
    print(df_sample_heating)

    # Create a sample dataframe for the waterHeating category
    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['waterHeating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'include_waterHeating', 'waterHeating_lifetime_fuelCost']
    )
    print(df_sample_waterHeating)

    # Create a sample dataframe for the clothesDrying category
    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['clothesDrying'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_lifetime_fuelCost']
    )
    print(df_sample_clothesDrying)

    # Create a sample dataframe for the cooking category
    df_sample_cooking = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_fuelCost']
    )
    print(df_sample_cooking)

# Calculate Capital Costs and Rebate Amounts

## Calculate Capital Costs (Applicable to All Scenarios)

In [ ]:
print("""
====================================================================================================================================================================
PRIVATE IMPACTS: NET CAPITAL COSTS AND TOTAL CAPITAL COSTS
====================================================================================================================================================================
- Step 1: Calculate annual operating (fuel) costs ----- DONE -----
- Step 2: Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
- Step 3: Calculate replacement cost (replacing existing piece of eqipment with similar technology)
- Step 4: Calculate net equipment capital costs
- Step 5: Calculate private NPV
====================================================================================================================================================================
""")

In [ ]:
# UPDATED MARCH 24, 2025 @ 4:30 PM - REMOVED RSMEANS CCI ADJUSTMENTS
from cmu_tare_model.utils.inflation_adjustment import *

# Collect Capital Cost Data for different End-uses
filename = "tare_retrofit_costs_cpi.xlsx"
relative_path = os.path.join("cmu_tare_model", "data", "retrofit_costs", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_heating_retrofit_costs = pd.read_excel(io=file_path, sheet_name='heating_costs')
df_waterHeating_retrofit_costs = pd.read_excel(io=file_path, sheet_name='waterHeating_costs')
df_clothesDrying_retrofit_costs = pd.read_excel(io=file_path, sheet_name='clothesDrying_costs')
df_cooking_retrofit_costs = pd.read_excel(io=file_path, sheet_name='cooking_costs')
df_enclosure_retrofit_costs = pd.read_excel(io=file_path, sheet_name='enclosure_upgrade_costs')

### Space Heating and No Enclosure Upgrade

#### Space Heating Capital Costs

In [ ]:
print("""
====================================================================================================================================================================
Capital Costs: Space Heating
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...
""")
# UPDATED MARCH 24, 2025 @ 4:30 PM - REMOVED RSMEANS CCI ADJUSTMENTS
from cmu_tare_model.private_impact.calculations.calculate_equipment_installation_costs import *
from cmu_tare_model.private_impact.calculations.calculate_equipment_replacement_costs import *

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
    'cost_per_kBtuh_progressive', 'cost_per_kBtuh_reference', 'cost_per_kBtuh_conservative',
    'otherCost_progressive', 'otherCost_reference', 'otherCost_conservative'
]

# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_heating_retrofit_costs[column] = round((df_heating_retrofit_costs[column] * df_heating_retrofit_costs['cpi_ratio'] * df_heating_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_heating_equipment_cost = df_heating_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')

# Call the function and obtain equipment specifications
print("Obtaining system specs ...")
df_euss_am_mp8_home = obtain_heating_system_specs(df=df_euss_am_mp8_home)

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Heat Pump for Space Heating (No Enclosure Upgrade) ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_heating_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='heating')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_heating_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='heating')

# Call the function and calculate installation premium based on existing housing characteristics
# calculate_heating_installation_premium(df, menu_mp, cpi_ratio_2023_2013)
print("Calculating Space Heating Specific Premiums (Ex: Removing Hydronic Boiler) ...")
df_euss_am_mp8_home = calculate_heating_installation_premium(df=df_euss_am_mp8_home,
                                                             menu_mp=menu_mp,
                                                             cpi_ratio_2023_2013=cpi_ratio_2023_2013)

# Display the df
print(df_euss_am_mp8_home)

### Water Heating

In [ ]:
print("""
====================================================================================================================================================================
Capital Costs: Water Heating
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...
""")

cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
    'cost_per_gallon_progressive', 'cost_per_gallon_reference', 'cost_per_gallon_conservative',
]

# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_waterHeating_retrofit_costs[column] = round((df_waterHeating_retrofit_costs[column] * df_waterHeating_retrofit_costs['cpi_ratio'] * df_waterHeating_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_waterHeating_equipment_cost = df_waterHeating_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_waterHeating_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Electric Heat Pump Water Heater ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_waterHeating_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='waterHeating')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_waterHeating_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='waterHeating')

# Display the df
print(df_euss_am_mp8_home)

### Clothes Drying

In [ ]:
print("""
====================================================================================================================================================================
Capital Costs: Clothes Drying
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ... 
""")

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
]
 
# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_clothesDrying_retrofit_costs[column] = round((df_clothesDrying_retrofit_costs[column] * df_clothesDrying_retrofit_costs['cpi_ratio'] * df_clothesDrying_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_clothesDrying_equipment_cost = df_clothesDrying_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_clothesDrying_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Ventless Heat Pump Clothes Dryer ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_clothesDrying_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='clothesDrying')


# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_clothesDrying_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='clothesDrying')

# Display the df
print(df_euss_am_mp8_home)

### Cooking

In [ ]:
print("""
====================================================================================================================================================================
Capital Costs: Cooking
====================================================================================================================================================================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...      
""")

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
]
 
# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_cooking_retrofit_costs[column] = round((df_cooking_retrofit_costs[column] * df_cooking_retrofit_costs['cpi_ratio'] * df_cooking_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_cooking_equipment_cost = df_cooking_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_cooking_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Electric Resistance Range ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_cooking_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='cooking')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_cooking_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='cooking')

# Display the df
print(df_euss_am_mp8_home)

 ## Calculate Rebate Amounts (Applicable to IRA-Reference)

In [ ]:
from cmu_tare_model.private_impact.data_processing.determine_rebate_eligibility_and_amount import *

# Determine Percent AMI and Rebate Amounts
# This needs to be done before running the calculate_percent_AMI function
df_euss_am_mp8_home = df_euss_am_mp8_home.copy()

# calculate_percent_AMI(df_results_IRA, df_county_medianIncome):
df_euss_am_mp8_home = calculate_percent_AMI(df_results_IRA=df_euss_am_mp8_home)

print("Calculating rebate amounts for Space Heating ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="heating",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Water Heating ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="waterHeating",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Clothes Drying ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="clothesDrying",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Cooking ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="cooking",
                                          menu_mp=menu_mp)

print(f"""
====================================================================================================================================================================
CALCULATE HOUSEHOLD PERCENT AREA MEDIAN INCOME (%AMI) AND REBATE ELIGIBILITY/AMOUNTS
====================================================================================================================================================================
determine_rebate_eligibility_and_amount.py file contains the function definitions for calculating rebate amounts and determining household %AMI.
process_income_data_for_rebates.py file contains additional information on data sources and procedures used to process data for determine_rebate_eligibility_and_amount.py file.

DATAFRAME: df_euss_am_mp8_home AFTER CALCULATING REBATE AMOUNTS
{df_euss_am_mp8_home}

====================================================================================================================================================================
""")

# SCENARIO ANALYSIS: Basic Pre-IRA Scenario
## - NREL End-Use Savings Shapes Database: Measure Package 8
## - AEO2023 No Inflation Reduction Act
## - Cambium 2021 MidCase Scenario

In [ ]:
# Measure Package 8
scenario_name = 'No Inflation Reduction Act'
cost_scenario = 'Fuel Costs: AEO2023 No Inflation Reduction Act'
grid_scenario = 'Electricity Grid: Cambium 2021 MidCase Scenario'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")

In [ ]:
from cmu_tare_model.public_impact.calculate_lifetime_public_impact_sensitivity import *

# # LAST UPDATED APRIL 13, 2025 @ 12:30PM
# def calculate_public_npv(
#     df: pd.DataFrame, 
#     df_baseline_climate: pd.DataFrame, 
#     df_baseline_health: pd.DataFrame,
#     df_mp_climate: pd.DataFrame,
#     df_mp_health: pd.DataFrame,
#     menu_mp: str, 
#     policy_scenario: str, 
#     rcm_model: str,
#     base_year: int = 2024,
#     discounting_method: str = 'public',
# ) -> pd.DataFrame:

# Create copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_mp8_home_ap2 = df_euss_am_mp8_home.copy()
df_euss_am_mp8_home_easiur = df_euss_am_mp8_home.copy()
df_euss_am_mp8_home_inmap = df_euss_am_mp8_home.copy()

# Example of how to call the updated calculate_public_npv function

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_public_npv(
    df=df_euss_am_mp8_home_ap2,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='AP2',
    base_year=2024,
    discounting_method='public'
)

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_public_npv(
    df=df_euss_am_mp8_home_easiur,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='EASIUR',
    base_year=2024,
    discounting_method='public'
)

# ============= INMAP =============
df_euss_am_mp8_home_inmap = calculate_public_npv(
    df=df_euss_am_mp8_home_inmap,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='InMAP',
    base_year=2024,
    discounting_method='public'
)

print(f"""  
====================================================================================================================================================================
PUBLIC PERSPECTIVE COSTS AND BENEFITS: NO INFLATION REDUCTION ACT
====================================================================================================================================================================
calculate_lifetime_public_impact.py file contains the definition for the calculate_public_npv function.
Additional information on emissions/damage factor lookups as well as marginal damages calculation methods can be found in the calculate_emissions_damages.py file. 
      
DATAFRAME FOR MP8 AFTER CALCULATING PUBLIC NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

In [ ]:
from cmu_tare_model.private_impact.calculate_lifetime_private_impact import *

# def calculate_private_NPV(
#         df: pd.DataFrame,
#         df_fuelCosts: pd.DataFrame,
#         input_mp: str,
#         menu_mp: int,
#         policy_scenario: str,
#         discounting_method: str = 'private_fixed',
#         base_year: int = 2024
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_private_NPV(
    df=df_euss_am_mp8_home_ap2,
    df_fuelCosts=df_mp8_noIRA_fuelCosts,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_private_NPV(
    df=df_euss_am_mp8_home_easiur,
    df_fuelCosts=df_mp8_noIRA_fuelCosts,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = calculate_private_NPV(
    df=df_euss_am_mp8_home_inmap,
    df_fuelCosts=df_mp8_noIRA_fuelCosts,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024
    )


print(f"""  
====================================================================================================================================================================
PRIVATE PERSPECTIVE COSTS AND BENEFITS: NO INFLATION REDUCTION ACT
====================================================================================================================================================================
calculate_lifetime_private_impact.py file contains the definition for the calculate_private_NPV function.
Additional information on fuel price lookups can be found in the calculate_fuel_costs.py file. 
      
DATAFRAME FOR MP8 AFTER CALCULATING PRIVATE NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

In [ ]:
from cmu_tare_model.adoption_potential.determine_adoption_potential_sensitivity import *

# def adoption_decision(df: pd.DataFrame,
#                       menu_mp: int,
#                       policy_scenario: str,
#                       rcm_model: str,
#                       cr_function: str,
#                       climate_sensitivity: bool = False  # Default is false because we use $190USD2020/mt in Joseph et al. (2025)
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='AP2',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='AP2',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='EASIUR',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='EASIUR',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='InMAP',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='InMAP',
    cr_function='h6c',
    climate_sensitivity=False
    )


print(f"""
====================================================================================================================================================================
ADOPTION FEASIBILITY OF VARIOUS RETROFITS: NO INFLATION REDUCTION ACT
====================================================================================================================================================================
determine_adoption_potential.py file contains the definition for the adoption_decision function.

DATAFRAME FOR MP8 AFTER DETERMINING ADOPTION FEASIBILITY: df_euss_am_mp8_home
      
AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

In [ ]:
# print(f"Adoption potential for AP2: {df_euss_am_mp8_home_ap2.columns.to_list()}")
# print(f"Adoption potential for EASIUR: {df_euss_am_mp8_home_easiur.columns.to_list()}")
# print(f"Adoption potential for InMAP: {df_euss_am_mp8_home_inmap.columns.to_list()}")


# Basic IRA-Reference Scenario:
## - NREL End-Use Savings Shapes Database: Measure Package 8
## - AEO2023 REFERENCE CASE - HDD and Fuel Price Projections
## - Cambium 2023 MidCase Scenario

In [ ]:
# Measure Package 8
scenario_name = 'Basic IRA-Reference'
cost_scenario = 'Fuel Costs: AEO2023 Reference Case'
grid_scenario = 'Electricity Grid: Cambium 2023 MidCase Scenario'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")

In [ ]:
# from cmu_tare_model.public_impact.calculate_lifetime_public_impact_sensitivity import *

# # LAST UPDATED APRIL 13, 2025 @ 12:30PM
# def calculate_public_npv(
#     df: pd.DataFrame, 
#     df_baseline_climate: pd.DataFrame, 
#     df_baseline_health: pd.DataFrame,
#     df_mp_climate: pd.DataFrame,
#     df_mp_health: pd.DataFrame,
#     menu_mp: str, 
#     policy_scenario: str, 
#     rcm_model: str,
#     base_year: int = 2024,
#     discounting_method: str = 'public',
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_public_npv(
    df=df_euss_am_mp8_home_ap2,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='AP2',
    base_year=2024,
    discounting_method='public'
)

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_public_npv(
    df=df_euss_am_mp8_home_easiur,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='EASIUR',
    base_year=2024,
    discounting_method='public'
)

# ============= INMAP =============
df_euss_am_mp8_home_inmap = calculate_public_npv(
    df=df_euss_am_mp8_home_inmap,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='InMAP',
    base_year=2024,
    discounting_method='public'
)

print(f"""  
====================================================================================================================================================================
PUBLIC PERSPECTIVE COSTS AND BENEFITS: AEO2023 Reference Case
====================================================================================================================================================================
calculate_lifetime_public_impact.py file contains the definition for the calculate_public_npv function.
Additional information on emissions/damage factor lookups as well as marginal damages calculation methods can be found in the calculate_emissions_damages.py file. 
      
DATAFRAME FOR MP8 AFTER CALCULATING PUBLIC NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

In [ ]:
# from cmu_tare_model.private_impact.calculate_lifetime_private_impact import *

# def calculate_private_NPV(
#         df: pd.DataFrame,
#         df_fuelCosts: pd.DataFrame,
#         input_mp: str,
#         menu_mp: int,
#         policy_scenario: str,
#         discounting_method: str = 'private_fixed',
#         base_year: int = 2024
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = calculate_private_NPV(
    df=df_euss_am_mp8_home_ap2,
    df_fuelCosts=df_mp8_IRA_fuelCosts,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = calculate_private_NPV(
    df=df_euss_am_mp8_home_easiur,
    df_fuelCosts=df_mp8_IRA_fuelCosts,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = calculate_private_NPV(
    df=df_euss_am_mp8_home_inmap,
    df_fuelCosts=df_mp8_IRA_fuelCosts,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024
    )


print(f"""  
====================================================================================================================================================================
PRIVATE PERSPECTIVE COSTS AND BENEFITS: AEO2023 Reference Case
====================================================================================================================================================================
calculate_lifetime_private_impact.py file contains the definition for the calculate_private_NPV function.
Additional information on fuel price lookups can be found in the calculate_fuel_costs.py file. 
      
DATAFRAME FOR MP8 AFTER CALCULATING PRIVATE NPV: df_euss_am_mp8_home

AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

In [ ]:
# from cmu_tare_model.adoption_potential.determine_adoption_potential_sensitivity import *

# def adoption_decision(df: pd.DataFrame,
#                       menu_mp: int,
#                       policy_scenario: str,
#                       rcm_model: str,
#                       cr_function: str,
#                       climate_sensitivity: bool = False  # Default is false because we use $190USD2020/mt in Joseph et al. (2025)
# ) -> pd.DataFrame:

# ============= AP2 =============
df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='AP2',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='AP2',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ EASIUR =============
df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='EASIUR',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='EASIUR',
    cr_function='h6c',
    climate_sensitivity=False
    )

# ============ InMAP =============
df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='InMAP',
    cr_function='acs',
    climate_sensitivity=False
    )

df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='InMAP',
    cr_function='h6c',
    climate_sensitivity=False
    )


print(f"""
====================================================================================================================================================================
ADOPTION FEASIBILITY OF VARIOUS RETROFITS: AEO2023 Reference Case
====================================================================================================================================================================
determine_adoption_potential.py file contains the definition for the adoption_decision function.

DATAFRAME FOR MP8 AFTER DETERMINING ADOPTION FEASIBILITY: df_euss_am_mp8_home
      
AP2: 
-----------------------------------------------
{df_euss_am_mp8_home_ap2}

EASIUR:
-----------------------------------------------
{df_euss_am_mp8_home_easiur}

InMAP:
-----------------------------------------------
{df_euss_am_mp8_home_inmap}
      
""")

In [ ]:
print(f"Adoption potential for AP2: {df_euss_am_mp8_home_ap2.columns.to_list()}")
print(f"Adoption potential for EASIUR: {df_euss_am_mp8_home_easiur.columns.to_list()}")
print(f"Adoption potential for InMAP: {df_euss_am_mp8_home_inmap.columns.to_list()}")


In [ ]:
df_euss_am_mp8_home_ap2

In [ ]:
df_euss_am_mp8_home_easiur

In [ ]:
df_euss_am_mp8_home_inmap

# Model Runtime

In [ ]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

In [ ]:
# SPACE HEATING: MULTI-INDEX DATAFRAMES
df_multiIndex_basic_heating_adoption_lrmer = create_multiIndex_adoption_df(df_basic_summary_heating, 8, 'heating', 'lrmer')
df_multiIndex_basic_heating_adoption_srmer = create_multiIndex_adoption_df(df_basic_summary_heating, 8, 'heating', 'srmer')

df_multiIndex_moderate_heating_adoption_lrmer = create_multiIndex_adoption_df(df_moderate_summary_heating, 9, 'heating', 'lrmer')
df_multiIndex_moderate_heating_adoption_srmer = create_multiIndex_adoption_df(df_moderate_summary_heating, 9, 'heating', 'srmer')

df_multiIndex_advanced_heating_adoption_lrmer = create_multiIndex_adoption_df(df_advanced_summary_heating, 10, 'heating', 'lrmer')
df_multiIndex_advanced_heating_adoption_srmer = create_multiIndex_adoption_df(df_advanced_summary_heating, 10, 'heating', 'srmer')

# WATER HEATING: MULTI-INDEX DATAFRAMES
df_multiIndex_basic_waterHeating_adoption_lrmer = create_multiIndex_adoption_df(df_basic_summary_waterHeating, 8, 'waterHeating', 'lrmer')
df_multiIndex_basic_waterHeating_adoption_srmer = create_multiIndex_adoption_df(df_basic_summary_waterHeating, 8, 'waterHeating', 'srmer')

df_multiIndex_moderate_waterHeating_adoption_lrmer = create_multiIndex_adoption_df(df_moderate_summary_waterHeating, 9, 'waterHeating', 'lrmer')
df_multiIndex_moderate_waterHeating_adoption_srmer = create_multiIndex_adoption_df(df_moderate_summary_waterHeating, 9, 'waterHeating', 'srmer')

df_multiIndex_advanced_waterHeating_adoption_lrmer = create_multiIndex_adoption_df(df_advanced_summary_waterHeating, 10, 'waterHeating', 'lrmer')
df_multiIndex_advanced_waterHeating_adoption_srmer = create_multiIndex_adoption_df(df_advanced_summary_waterHeating, 10, 'waterHeating', 'srmer')

# CLOTHES DRYING: MULTI-INDEX DATAFRAMES
df_multiIndex_basic_clothesDrying_adoption_lrmer = create_multiIndex_adoption_df(df_basic_summary_clothesDrying, 8, 'clothesDrying', 'lrmer')
df_multiIndex_basic_clothesDrying_adoption_srmer = create_multiIndex_adoption_df(df_basic_summary_clothesDrying, 8, 'clothesDrying', 'srmer')

df_multiIndex_moderate_clothesDrying_adoption_lrmer = create_multiIndex_adoption_df(df_moderate_summary_clothesDrying, 9, 'clothesDrying', 'lrmer')
df_multiIndex_moderate_clothesDrying_adoption_srmer = create_multiIndex_adoption_df(df_moderate_summary_clothesDrying, 9, 'clothesDrying', 'srmer')

df_multiIndex_advanced_clothesDrying_adoption_lrmer = create_multiIndex_adoption_df(df_advanced_summary_clothesDrying, 10, 'clothesDrying', 'lrmer')
df_multiIndex_advanced_clothesDrying_adoption_srmer = create_multiIndex_adoption_df(df_advanced_summary_clothesDrying, 10, 'clothesDrying', 'srmer')

# COOKING: MULTI-INDEX DATAFRAMES
df_multiIndex_basic_cooking_adoption_lrmer = create_multiIndex_adoption_df(df_basic_summary_cooking, 8, 'cooking', 'lrmer')
df_multiIndex_basic_cooking_adoption_srmer = create_multiIndex_adoption_df(df_basic_summary_cooking, 8, 'cooking', 'srmer')

df_multiIndex_moderate_cooking_adoption_lrmer = create_multiIndex_adoption_df(df_moderate_summary_cooking, 9, 'cooking', 'lrmer')
df_multiIndex_moderate_cooking_adoption_srmer = create_multiIndex_adoption_df(df_moderate_summary_cooking, 9, 'cooking', 'srmer')

df_multiIndex_advanced_cooking_adoption_lrmer = create_multiIndex_adoption_df(df_advanced_summary_cooking, 10, 'cooking', 'lrmer')
df_multiIndex_advanced_cooking_adoption_srmer = create_multiIndex_adoption_df(df_advanced_summary_cooking, 10, 'cooking', 'srmer')

In [ ]:
df_multiIndex_basic_heating_adoption_lrmer

In [ ]:
# LRMER
# SPACE HEATING - ALL 3 SCENARIOS
# Function call remains as before
subplot_grid_adoption_vBar(
    dataframes=[df_multiIndex_basic_heating_adoption_lrmer, 
                df_multiIndex_moderate_heating_adoption_lrmer, 
                df_multiIndex_advanced_heating_adoption_lrmer,
               ],
    scenarios_list = [
        ['preIRA_mp8_heating_adoption_lrmer', 'iraRef_mp8_heating_adoption_lrmer'],
        ['preIRA_mp9_heating_adoption_lrmer', 'iraRef_mp9_heating_adoption_lrmer'],
        ['preIRA_mp10_heating_adoption_lrmer', 'iraRef_mp10_heating_adoption_lrmer']
    ],
    subplot_positions=[(0, 0), (0, 1), (0, 2)],
    # x_labels=["", "",
    #           "", "",
    #           "", "Household Income Designation and Baseline Fuel (Pre-Retrofit)",
    #          ],
    plot_titles=["Air Source Heat Pump (ASHP):\nPre-IRA vs. IRA-Reference", "ASHP + Basic Enclosure Upgrade:\nPre-IRA vs. IRA-Reference", "ASHP + Enhanced Enclosure Upgrade:\nPre-IRA vs. IRA-Reference"],
    y_labels=["LRMER - Retrofit Adoption Potential (%)", "", "",
             ],
    figure_size=(20,10),
    sharex=True,
    sharey=True,
#     filter_fuel=['Natural Gas', 'Electricity', 'Propane']
)

# SRMER
# SPACE HEATING - ALL 3 SCENARIOS
# Function call remains as before
subplot_grid_adoption_vBar(
    dataframes=[df_multiIndex_basic_heating_adoption_srmer, 
                df_multiIndex_moderate_heating_adoption_srmer, 
                df_multiIndex_advanced_heating_adoption_srmer,
               ],
    scenarios_list = [
        ['preIRA_mp8_heating_adoption_srmer', 'iraRef_mp8_heating_adoption_srmer'],
        ['preIRA_mp9_heating_adoption_srmer', 'iraRef_mp9_heating_adoption_srmer'],
        ['preIRA_mp10_heating_adoption_srmer', 'iraRef_mp10_heating_adoption_srmer']
    ],
    subplot_positions=[(0, 0), (0, 1), (0, 2)],
    # x_labels=["", "",
    #           "", "",
    #           "", "Household Income Designation and Baseline Fuel (Pre-Retrofit)",
    #          ],
    plot_titles=["Air Source Heat Pump (ASHP):\nPre-IRA vs. IRA-Reference", "ASHP + Basic Enclosure Upgrade:\nPre-IRA vs. IRA-Reference", "ASHP + Enhanced Enclosure Upgrade:\nPre-IRA vs. IRA-Reference"],
    y_labels=["SRMER - Retrofit Adoption Potential (%)", "", "",
             ],
    figure_size=(20,10),
    sharex=True,
    sharey=True,
#     filter_fuel=['Natural Gas', 'Electricity', 'Propane']
)

In [ ]:
df_multiIndex_basic_heating_adoption_lrmer

In [ ]:
df_multiIndex_basic_heating_adoption_srmer

In [ ]:
df_multiIndex_moderate_heating_adoption_lrmer

In [ ]:
df_multiIndex_moderate_heating_adoption_srmer

In [ ]:
df_multiIndex_advanced_heating_adoption_lrmer

In [ ]:
df_multiIndex_advanced_heating_adoption_srmer

# Water Heating, Clothes Drying, and Cooking

In [ ]:
# Function call remains as before
subplot_grid_adoption_vBar(
    dataframes=[df_multiIndex_basic_waterHeating_adoption_lrmer, 
                df_multiIndex_basic_clothesDrying_adoption_lrmer, 
                df_multiIndex_basic_cooking_adoption_lrmer,
               ],
    scenarios_list = [
        ['preIRA_mp8_waterHeating_adoption_lrmer', 'iraRef_mp8_waterHeating_adoption_lrmer'],
        ['preIRA_mp8_clothesDrying_adoption_lrmer', 'iraRef_mp8_clothesDrying_adoption_lrmer'],
        ['preIRA_mp8_cooking_adoption_lrmer', 'iraRef_mp8_cooking_adoption_lrmer']
    ],
    subplot_positions=[(0, 0), (0, 1), (0, 2),
                    #    (1, 0), (1, 1), (1, 2),
                    #    (2, 0), (2, 1), (2, 2)
                       ],
    x_labels=["", "", "",
              "", "", "",
              "", "Household Income Designation and Baseline Fuel (Pre-Retrofit)", ""
             ],
    plot_titles=["Heat Pump Water Heater:\nPre-IRA vs. IRA-Reference", "Heat Pump Clothes Dryer:\nPre-IRA vs. IRA-Reference", "Electric Resistance Range:\nPre-IRA vs. IRA-Reference"],
    y_labels=["LRMER - Retrofit Adoption Potential (%)", "", "",
             ],
    figure_size=(20,10),
    sharex=False,
    sharey=True,
#     filter_fuel=['Natural Gas', 'Electricity', 'Propane']
)

# Function call remains as before
subplot_grid_adoption_vBar(
    dataframes=[df_multiIndex_basic_waterHeating_adoption_srmer, 
                df_multiIndex_basic_clothesDrying_adoption_srmer, 
                df_multiIndex_basic_cooking_adoption_srmer,
               ],
    scenarios_list = [
        ['preIRA_mp8_waterHeating_adoption_srmer', 'iraRef_mp8_waterHeating_adoption_srmer'],
        ['preIRA_mp8_clothesDrying_adoption_srmer', 'iraRef_mp8_clothesDrying_adoption_srmer'],
        ['preIRA_mp8_cooking_adoption_srmer', 'iraRef_mp8_cooking_adoption_srmer']
    ],
    subplot_positions=[(0, 0), (0, 1), (0, 2),
                    #    (1, 0), (1, 1), (1, 2),
                    #    (2, 0), (2, 1), (2, 2)
                       ],
    x_labels=["", "", "",
              "", "", "",
              "", "Household Income Designation and Baseline Fuel (Pre-Retrofit)", ""
             ],
    plot_titles=["Heat Pump Water Heater:\nPre-IRA vs. IRA-Reference", "Heat Pump Clothes Dryer:\nPre-IRA vs. IRA-Reference", "Electric Resistance Range:\nPre-IRA vs. IRA-Reference"],
    y_labels=["SRMER - Retrofit Adoption Potential (%)", "", "",
             ],
    figure_size=(20,10),
    sharex=False,
    sharey=True,
#     filter_fuel=['Natural Gas', 'Electricity', 'Propane']
)

In [ ]:
df_multiIndex_basic_waterHeating_adoption_lrmer

In [ ]:
df_multiIndex_basic_waterHeating_adoption_srmer

In [ ]:
df_multiIndex_basic_clothesDrying_adoption_lrmer

In [ ]:
df_multiIndex_basic_clothesDrying_adoption_srmer

In [ ]:
df_multiIndex_basic_cooking_adoption_lrmer

In [ ]:
df_multiIndex_basic_cooking_adoption_srmer